# A short tutorial for the `prototorch.models` plugin

## Introduction

This is a short tutorial for the [models](https://github.com/si-cim/prototorch_models) plugin of the [ProtoTorch](https://github.com/si-cim/prototorch) framework.

[ProtoTorch](https://github.com/si-cim/prototorch) provides [torch.nn](https://pytorch.org/docs/stable/nn.html) modules and utilities to implement prototype-based models. However, it is up to the user to put these modules together into models and handle the training of these models. Expert machine-learning practioners and researchers sometimes prefer this level of control. However, this leads to a lot of boilerplate code that is essentially same across many projects. Needless to say, this is a source of a lot of frustration. [PyTorch-Lightning](https://pytorch-lightning.readthedocs.io/en/latest/) is a framework that helps avoid a lot of this frustration by handling the boilerplate code for you so you don't have to reinvent the wheel every time you need to implement a new model.

With the [prototorch.models](https://github.com/si-cim/prototorch_models) plugin, we've gone one step further and pre-packaged commonly used prototype-models like GMLVQ as [Lightning-Modules](https://pytorch-lightning.readthedocs.io/en/latest/api/pytorch_lightning.core.lightning.html?highlight=lightning%20module#pytorch_lightning.core.lightning.LightningModule). With only a few lines to code, it is now possible to build and train prototype-models. It quite simply cannot get any simpler than this.

## Basics

First things first. When working with the models plugin, you'll probably need `torch`, `prototorch` and `pytorch_lightning`. So, we recommend that you import all three like so:

In [ ]:
import prototorch as pt
import pytorch_lightning as pl
import torch

### Building Models

Let's start by building a `GLVQ` model. It is one of the simplest models to build. The only requirements are a prototype distribution and an initializer.

In [ ]:
model = pt.models.GLVQ(
    hparams=dict(distribution=[1, 1, 1]),
    prototypes_initializer=pt.initializers.ZerosCompInitializer(2),
)

In [ ]:
print(model)

The key `distribution` in the `hparams` argument describes the prototype distribution. If it is a Python [list](https://docs.python.org/3/tutorial/datastructures.html), it is assumed that there are as many entries in this list as there are classes, and the number at each location of this list describes the number of prototypes to be used for that particular class. So, `[1, 1, 1]` implies that we have three classes with one prototype per class.   If it is a Python [tuple](https://docs.python.org/3/tutorial/datastructures.html), a shorthand of `(num_classes, prototypes_per_class)` is assumed. If it is a Python [dictionary](https://docs.python.org/3/tutorial/datastructures.html), the key-value pairs describe the class label and the number of prototypes for that class respectively. So, `{0: 2, 1: 2, 2: 2}` implies that we have three classes with labels `{1, 2, 3}`, each equipped with two prototypes. If however, the dictionary contains the keys `"num_classes"` and `"per_class"`, they are parsed to use their values as one might expect.

The `prototypes_initializer` argument describes how the prototypes are meant to be initialized. This argument has to be an instantiated object of some kind of [AbstractComponentsInitializer](https://github.com/si-cim/prototorch/blob/dev/prototorch/components/initializers.py#L18). If this is a [ShapeAwareCompInitializer](https://github.com/si-cim/prototorch/blob/dev/prototorch/components/initializers.py#L41), this only requires a `shape` arugment that describes the shape of the prototypes. So, `pt.initializers.ZerosCompInitializer(3)` creates 3d-vector prototypes all initialized to zeros.

### Data

The preferred way to working with data in `torch` is to use the [Dataset and Dataloader API](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html). There a few pre-packaged datasets available under `prototorch.datasets`. See [here](https://prototorch.readthedocs.io/en/latest/api.html#module-prototorch.datasets) for a full list of available datasets.

In [ ]:
train_ds = pt.datasets.Iris(dims=[0, 2])

In [ ]:
type(train_ds)

In [ ]:
train_ds.data.shape, train_ds.targets.shape

Once we have such a dataset, we could wrap it in a `Dataloader` to load the data in batches, and possibly apply some transformations on the fly.

In [ ]:
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=2)

In [ ]:
type(train_loader)

In [ ]:
x_batch, y_batch = next(iter(train_loader))
print(f"{x_batch=}, {y_batch=}")

This perhaps seems like a lot of work for a small dataset that fits completely in memory. However, this comes in very handy when dealing with huge datasets that can only be processed in batches.

### Training

If you're familiar with other deep learning frameworks, you might perhaps expect a `.fit(...)` or `.train(...)` method. However, in PyTorch-Lightning, this is done slightly differently. We first create a trainer and then pass the model and the Dataloader to `trainer.fit(...)` instead. So, it is more functional in style than object-oriented.

In [ ]:
trainer = pl.Trainer(max_epochs=2, weights_summary=None)

In [ ]:
trainer.fit(model, train_loader)

### From data to a trained model - a very minimal example

In [ ]:
train_ds = pt.datasets.Iris(dims=[0, 2])
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=32)

model = pt.models.GLVQ(
    dict(distribution=(3, 2), lr=0.1),
    prototypes_initializer=pt.initializers.SMCI(train_ds),
)

trainer = pl.Trainer(max_epochs=50, weights_summary=None)
trainer.fit(model, train_loader)

### Saving/Loading trained models

Pytorch Lightning can automatically checkpoint the model during various stages of training, but it also possible to manually save a checkpoint after training.

In [ ]:
ckpt_path = "./checkpoints/glvq_iris.ckpt"
trainer.save_checkpoint(ckpt_path)

In [ ]:
loaded_model = pt.models.GLVQ.load_from_checkpoint(ckpt_path, strict=False)

### Visualizing decision boundaries in 2D

In [ ]:
pt.models.VisGLVQ2D(data=train_ds).visualize(loaded_model)

### Saving/Loading trained weights

In most cases, the checkpointing workflow is sufficient. In some cases however, one might want to only save the trained weights from the model. The disadvantage of this method is that the model has be re-created using compatible initialization parameters before the weights could be loaded.

In [ ]:
ckpt_path = "./checkpoints/glvq_iris_weights.pth"
torch.save(model.state_dict(), ckpt_path)

In [ ]:
model = pt.models.GLVQ(
    dict(distribution=(3, 2)),
    prototypes_initializer=pt.initializers.ZerosCompInitializer(2),
)

In [ ]:
pt.models.VisGLVQ2D(data=train_ds, title="Before loading the weights").visualize(model)

In [ ]:
torch.load(ckpt_path)

In [ ]:
model.load_state_dict(torch.load(ckpt_path), strict=False)

In [ ]:
pt.models.VisGLVQ2D(data=train_ds, title="After loading the weights").visualize(model)

## Advanced

### Initializing prototypes with a subset of a dataset (along with transformations)

In [ ]:
import prototorch as pt
import pytorch_lightning as pl
import torch
from torchvision import transforms
from torchvision.datasets import MNIST

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
train_ds = MNIST(
    "~/datasets",
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomHorizontalFlip(p=1.0),
        transforms.RandomVerticalFlip(p=1.0),
        transforms.ToTensor(),
    ]),
)

In [ ]:
s = int(0.05 * len(train_ds))
init_ds, rest_ds = torch.utils.data.random_split(train_ds, [s, len(train_ds) - s])

In [ ]:
init_ds

In [ ]:
model = pt.models.ImageGLVQ(
    dict(distribution=(10, 5)),
    prototypes_initializer=pt.initializers.SMCI(init_ds),
)

In [ ]:
plt.imshow(model.get_prototype_grid(num_columns=10))

## FAQs

### How do I Retrieve the prototypes and their respective labels from the model?

For prototype models, the prototypes can be retrieved (as `torch.tensor`) as `model.prototypes`. You can convert it to a NumPy Array by calling `.numpy()` on the tensor if required.

```python
>>> model.prototypes.numpy()
```

Similarly, the labels of the prototypes can be retrieved via `model.prototype_labels`.

```python
>>> model.prototype_labels
```

### How do I make inferences/predictions/recall with my trained model?

The models under [prototorch.models](https://github.com/si-cim/prototorch_models) provide a `.predict(x)` method for making predictions. This returns the predicted class labels. It is essential that the input to this method is a `torch.tensor` and not a NumPy array. Model instances are also callable. So, you could also just say `model(x)` as if `model` were just a function. However, this returns a (pseudo)-probability distribution over the classes.

#### Example

```python
>>> y_pred = model.predict(torch.Tensor(x_train))  # returns class labels
```
or, simply
```python
>>> y_pred = model(torch.Tensor(x_train))  # returns probabilities
```